In [0]:
with CTE AS (
  SELECT
    u.custom_tags,
    SUM(u.usage_quantity * p.pricing.default) AS total
  FROM
    system.billing.usage u
      RIGHT JOIN system.billing.list_prices p
        ON u.sku_name = p.sku_name
        AND (
          (u.usage_start_time BETWEEN p.price_start_time AND p.price_end_time)
          OR (
            (u.usage_start_time > p.price_start_time)
            AND (p.price_end_time IS NULL)
          )
        )
  WHERE
      (map_contains_key(u.custom_tags, 'tests') OR map_contains_key(u.custom_tags, 'test')) AND
    (u.usage_start_time BETWEEN timestamp'2025-04-27T21:40:00' AND timestamp'2025-04-28T06:00:00') -- test period
  GROUP BY
    u.custom_tags
  ORDER BY
    total ASC
)
SELECT
  concat_ws(': ', row_number() OVER (ORDER BY total ASC), coalesce(custom_tags['tests'], custom_tags['test'])) as rank,
  total
FROM
  CTE

Databricks visualization. Run in Databricks to view.